<a href="https://colab.research.google.com/github/duvvusuribabu/student-dropout-prediction-/blob/main/ADS_HACKATHON_16_8_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ADVANCE DATA SCIENCE HACKATHON - (16/08/22 - 17/08/22)


---



TEAM : CODE BUDDIES

 ##  Prediction of Student Dropouts


> *Importing the required libraries*

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

> *Loading the dataset*

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/student-data.csv')
data.head(3)

## Exploratory Data Analysis

In [ ]:
data.shape

From above, it is clear that the dataset contains `31` features with `395` records.

In [ ]:
print('Total number of students : {}'.format(data.shape[0]))

print('Total number of students  passed: {}'.format(len(data[data.passed=='yes'])))

print('Total number of students  failed: {}'.format(len(data[data.passed=='no'])))

print('Pass percentage: {}%'.format(round(len(data[data.passed=='yes'])/data.shape[0],2)*100))

In [ ]:
data.info()

*From the above output we concluded that there are no null values but there are some features that has non int values. These values have to converted to numerical categorical data.*

> *Convertion of values of Object type to Integer type:*




In [ ]:
for i in data.columns:
  print(i,set(data[i].values))

The above code helps us to have a glance at the unique values present in each feature of the dataset.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# objCols = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'passed']
# data[objCols] = data[objCols].apply(le.fit_transform)

# objCols=[]
# for name in feature_names:
#   if data[name].dtype =='object':
#     objCols.append(name)
# data[objCols] = data[objCols].apply(le.fit_transform)


feature_names = data.columns.values

for name in feature_names:
  if data[name].dtype =='object':
    data[name] = le.fit_transform(data[name])
data.head()


*Now we find that the data set is transformed into a dataset with only integer values.*

*As we know that the model can't be trained on the categorical data, so we converted the values of object type into integer type.*

In [ ]:
# sns.catplot(x="passed", kind="count", data=data)
sns.countplot('passed', data=data)
plt.title('Passed?')

In [ ]:
Passed_data =[data.passed.value_counts()]

print("Total passed(1): {}".format(Passed_data[0][1]))
print("Pass Percentage: {}\n".format(round(Passed_data[0][1]*100/(Passed_data[0][0]+Passed_data[0][1]),2)))

print("Total Dropped Out(0): {}".format(Passed_data[0][0]))
print("Dropout Percentage Percentage: {}".format(round(Passed_data[0][0]*100/(Passed_data[0][0]+Passed_data[0][1]),2)))

*For optimal/high accuracy, a good dataset must not contain features with high correaltion among them.
The `.corr()` method of pandas library helps us to find the co-relation among the features.*

In [ ]:
corMatrix = data.corr().abs()
corMatrix

*Since the upper triangle and lower triangle matrices are same, we shall consider any one of them.*

In [ ]:
    upper_matrix = corMatrix.where(np.triu(np.ones(corMatrix.shape),k=1).astype(np.bool_))
    upper_matrix.head()

In [ ]:
drop_columns = [col for col in upper_matrix if any(upper_matrix[col]>0.65)]
print(drop_columns)

*The features of the dataset are not highly co-related. Hence, no features are required to be removed.*

In [20]:
X = data.drop('passed',axis=1)
y= data.passed

In [21]:
# shuffled_df = data.sample(frac=1,random_state=4)

# dropped_df = shuffled_df.loc[shuffled_df['passed'] == 0]
# passed_df = shuffled_df.loc[shuffled_df['passed'] == 1].sample(n=130,random_state=42)

# normalized_df = pd.concat([dropped_df, passed_df])

# plt.figure(figsize=(8, 8))
# sns.countplot('passed', data=normalized_df)a
# plt.title('Balanced Classes')
# plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

sel = SelectFromModel(DecisionTreeClassifier())
sel.fit(X_train, y_train)

sel.get_support()
selected_feat= X_train.columns[(sel.get_support())]
selected_feat

In [ ]:
df = pd.DataFrame(data=data, columns=selected_feat)
df_target = data['passed']
# df1 = pd.concat([df, df_target], ignore_index=True, sort=False)
df = df.join(df_target, lsuffix='_caller', rsuffix='_other')
df.head()

In [24]:
X = df.drop('passed',axis=1)
y = df.passed

In [25]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

# rus = RandomUnderSampler() 
# X, y = rus.fit_resample(X, y)


ros = RandomOverSampler()
X, y = ros.fit_resample(X, y)


# sampler = SMOTE()
# X, y = sampler.fit_resample(X, y)

# # data.isnull().sum()

# X = X_ros
# y = y_ros

In [ ]:
df.corr()

##Data Visualization

*By using pandas, matplotlib and seaborn libraries we can visualize the data graphically and can understand more clearly.*

In [ ]:
df.hist(edgecolor='black',bins = 25, figsize= (20,20))
plt.show()

*The `hist()` method of pandas is used to represent the histograms of each feature specified in the data frame.*

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(df.corr(), cmap="YlGnBu", annot=True)

> Splitting data for Training and Testing

In [29]:
#splitting the dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

> Training the models and Evaluating their performance

*Since the target feature is categorical, the Machine learning models that are used to train and predict on this dataset should be of type Classification.*

*The various metrics for evaluating Classification models are*




> Confusion matrix
 *   Accuracy
 *   Recall
 *   Precision
 *   F1_score
 
> Classification Report







In [30]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [31]:
scores_df=pd.DataFrame({'Model':[], 'F1_score':[], 'Accuracy':[]})

### Logistic Regression

*`Logistic Regression` is used for predicting the categorical dependent variable using a given set of independent variables.*

In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression


model = LogisticRegression(C=0.1, max_iter=250)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
# print('{}\'s Accuracy :{}'.format(type(model).__name__,accuracy_score(y_test, y_pred)))
print('{}\'s F1_Score :{}'.format(type(model).__name__,f1_score(y_test, y_pred)))

scores_df = scores_df.append(pd.DataFrame({'Model':[type(model).__name__],
                               'F1_score':[f1_score(y_test, y_pred)],
                               'Accuracy':[accuracy_score(y_test, y_pred)]
                               }), ignore_index=True)

The Accuracy of LogisticRegression is 0.72 and F1_score is 0.82

### Naive Bayes ( BernoulliNB )

*`Naïve Bayes` algorithm is a supervised learning algorithm, which is based on Bayes theorem and used for solving classification problems.
It is a probabilistic classifier, which means it predicts on the basis of the probability of an object.*

In [ ]:
#naive bayes
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
# print('{}\'s Accuracy :{}'.format(type(model).__name__,accuracy_score(y_test, y_pred)))
print('{}\'s F1_Score :{}'.format(type(model).__name__,f1_score(y_test, y_pred)))

scores_df = scores_df.append(pd.DataFrame({'Model':[type(model).__name__],
                               'F1_score':[f1_score(y_test, y_pred)],
                               'Accuracy':[accuracy_score(y_test, y_pred)]
                               }), ignore_index=True)

The Accuracy of Bernoulli Naive Bayes Classifier is 0.75 and F1_score is 0.84

### K Nearest Neighbour

*`K-NN` algorithm assumes the similarity between the new case/data and available cases and put the new case into the category that is most similar to the available categories.*

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=7, algorithm='auto')
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
confusion_matrix(y_test, y_pred)
# print('{}\'s Accuracy :{}'.format(type(model).__name__,accuracy_score(y_test, y_pred)))
print('{}\'s F1_Score :{}'.format(type(model).__name__,f1_score(y_test, y_pred)))

scores_df = scores_df.append(pd.DataFrame({'Model':[type(model).__name__],
                               'F1_score':[f1_score(y_test, y_pred)],
                               'Accuracy':[accuracy_score(y_test, y_pred)]
                               }), ignore_index=True)

The Accuracy of KNN Classifier is 0.68 and F1_score is 0.81

### Decision Tree

*`Decision Tree` is a tree-structured classifier, where internal nodes represent the features of a dataset, branches represent the decision rules and each leaf node represents the outcome.*

In [ ]:
#decision tree

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion='gini')
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
# print('{}\'s Accuracy :{}'.format(type(model).__name__,accuracy_score(y_test, y_pred)))
print('{}\'s F1_Score :{}'.format(type(model).__name__,f1_score(y_test, y_pred)))

scores_df = scores_df.append(pd.DataFrame({'Model':[type(model).__name__],
                               'F1_score':[f1_score(y_test, y_pred)],
                               'Accuracy':[accuracy_score(y_test, y_pred)]
                               }), ignore_index=True)

The Accuracy of Decision Tree Classifier is 0.68 and F1_score is 0.77

### Support Vector Machine

*`SVM` algorithm creates the best line or decision boundary that can segregate n-dimensional space into classes so that we can easily put the new data point in the correct category in the future. This best decision boundary is called a hyperplane.*

In [ ]:
# svm

from sklearn.svm import SVC

model = SVC(C=0.1)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
confusion_matrix(y_test, y_pred)
# print('{}\'s Accuracy :{}'.format(type(model).__name__,accuracy_score(y_test, y_pred)))
print('{}\'s F1_Score :{}'.format(type(model).__name__,f1_score(y_test, y_pred)))

scores_df = scores_df.append(pd.DataFrame({'Model':[type(model).__name__],
                               'F1_score':[f1_score(y_test, y_pred)],
                               'Accuracy':[accuracy_score(y_test, y_pred)]
                               }), ignore_index=True)

The Accuracy of Support Vector Machine Classifier is 0.71 and F1_score is 0.83

### Random Forest Classifier

*`Random Forest` is a classifier that contains a number of decision trees on various subsets of the given dataset and takes the average to improve the predictive accuracy of that dataset.*

In [ ]:
#random forest

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=13, criterion='gini',max_depth=10, max_features='auto')
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
confusion_matrix(y_test, y_pred)
# print('{}\'s Accuracy :{}'.format(type(model).__name__,accuracy_score(y_test, y_pred)))
print('{}\'s F1_Score :{}'.format(type(model).__name__,f1_score(y_test, y_pred)))

scores_df = scores_df.append(pd.DataFrame({'Model':[type(model).__name__],
                               'F1_score':[f1_score(y_test, y_pred)],
                               'Accuracy':[accuracy_score(y_test, y_pred)]
                               }), ignore_index=True)

The Accuracy of Random Forest Classifier is 0.75 and F1_score is 0.83

*Here we are using `Yellowbrick` for visualization of model prediction*

In [ ]:
classes = [1,0]
from yellowbrick.classifier import ConfusionMatrix
visualizer = ConfusionMatrix(model, classes=classes, support=True) 
visualizer.fit(X_train, y_train) 
visualizer.score(X_test, y_test)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
from yellowbrick.classifier import ClassificationReport
visualizer = ClassificationReport(model, classes=classes, support=True) 
visualizer.fit(X_train, y_train) 
visualizer.score(X_test, y_test)
plt.show()

In [ ]:
plt.scatter(y_test,y_pred)
plt.xlabel("Predicted Y")
plt.ylabel("Actual Y")
plt.title("Predicted against Actual plot")

### XGBOOST

In [41]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

numeric_pipeline = Pipeline(
    steps=[("impute", SimpleImputer(strategy="mean")), 
           ("scale", StandardScaler())]
)
num_cols = X.select_dtypes(include="number").columns

full_processor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_pipeline, num_cols),
    ]
)

X_processed = full_processor.fit_transform(X)
y_processed = SimpleImputer(strategy="most_frequent").fit_transform(
    y.values.reshape(-1, 1)
)

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=50
)

model = xgb.XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))
# print('{}\'s Accuracy :{}'.format(type(model).__name__,round(accuracy_score(y_test, y_pred),2)))
print('{}\'s F1_Score :{}'.format(type(model).__name__,f1_score(y_test, y_pred)))

scores_df = scores_df.append(pd.DataFrame({'Model':[type(model).__name__],
                               'F1_score':[f1_score(y_test, y_pred)],
                               'Accuracy':[accuracy_score(y_test, y_pred)]
                               }), ignore_index=True)

The Accuracy of XGBOOST Classifier is 0.72 and F1_score is 0.81

Since the data is unbalanced F1_score is considered superior to model accuracy in determining the better ML algorithm.

Therefore, the Accuracy and F1_Score of different models stated above are

In [ ]:
scores_df

###**Artificail Neural Networks**

*On the other hand we can also use the ann(artificial neural networks) models to perform the classification as well as Regression problems.*

>Importing the required libraries

In [44]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [45]:
model = Sequential()
model.add(Dense(12, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [46]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=5)

In [48]:
y_pred = model.predict(X_test)

5/5 [==============================] - 0s 3ms/step


## Conclusion


---


Among the above classifiers `Random Forest Classifier` performs well on this dataset. It has higher values in terms of both Accuracy - 80% and F1 Score - 0.85(approximately)

The deployment of this model can be seen here - [LINK](https://dropout-predictor.herokuapp.com)